## Loading Data - Docker

Let us understand how you can take care of loading data into Postgres Database running using Docker Container.

In [1]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/BUvelySsHDU?rel=0&amp;controls=1&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>

* Make sure database is created along with the user with right permissions. Also the user who want to use `COPY` command need to have **pg_read_server_files** role assigned.
  * Create file with sample data
  * Copy file into Docker container
  * Connect to Database
  * Create the table
  * Run `COPY` Command to copy the data.

**Prepare Data**

We need to create file with sample data and copy the files into the container.

* Sample File
In this case data is added to users.csv under **~/sms_db**.

```text
user_first_name,user_last_name,user_email_id,user_role,created_dt
Gordan,Bradock,gbradock0@barnesandnoble.com,A,2020-01-10
Tobe,Lyness,tlyness1@paginegialle.it,U,2020-02-10
Addie,Mesias,amesias2@twitpic.com,U,2020-03-05
Corene,Kohrsen,ckohrsen3@buzzfeed.com,U,2020-04-15
Darill,Halsall,dhalsall4@intel.com,U,2020-10-10
```
* Copy data

```shell
docker cp ~/sms_db/users.csv itv_pg:/tmp
```

**Create Database**

Here are the steps to create database.
* Connect to database as super user **postgres**

```shell
docker exec -it itv_pg psql -U postgres
```

* Create the database with right permissions.

```sql
CREATE DATABASE itversity_sms_db;
CREATE USER itversity_sms_user WITH PASSWORD 'sms_password';
GRANT ALL ON DATABASE itversity_sms_db TO itversity_sms_user;
GRANT pg_read_server_files TO itversity_sms_user;
```

* Exit using `\q`

**Connect to Database**

Use this command to connect to the newly created database.

```shell
psql -U itversity_sms_user \
  -h localhost \
  -p 5433 \
  -d itversity_sms_db \
  -W
```

**Create Table**

Here is the script to create the table.

```sql
CREATE TABLE users (
    user_id SERIAL PRIMARY KEY,
    user_first_name VARCHAR(30) NOT NULL,
    user_last_name VARCHAR(30) NOT NULL,
    user_email_id VARCHAR(50) NOT NULL,
    user_email_validated BOOLEAN DEFAULT FALSE,
    user_password VARCHAR(200),
    user_role VARCHAR(1) NOT NULL DEFAULT 'U', --U and A
    is_active BOOLEAN DEFAULT FALSE,
    created_dt DATE DEFAULT CURRENT_DATE
);
```

**Load Data**

Here are the steps to load and validate the data using `psql`.

* Load data using `COPY` Command

```shell
COPY users(user_first_name, user_last_name, 
    user_email_id, user_role, created_dt
) FROM '/tmp/users.csv'
DELIMITER ','
CSV HEADER;
```

* Validate by running queries

```sql
SELECT * FROM users;
```

## My CLI logs

```shell
[tharindu@acer-lap data-engineering-spark]$ docker ps -a
CONTAINER ID   IMAGE                                      COMMAND                  CREATED        STATUS                        PORTS                                       NAMES
c68b18841ca0   postgres                                   "docker-entrypoint.s…"   2 days ago     Exited (0) 10 hours ago                                                   my_pg
304414f98a65   itversity/itvdelab                         "/deploy.sh"             4 days ago     Exited (137) 33 minutes ago                                               data-engineering-spark_itvdelab_1
9bee507f35b3   postgres:13                                "docker-entrypoint.s…"   4 days ago     Exited (0) 4 days ago                                                     data-engineering-spark_cluster_util_db_1
8dec6c8c1b48   data-engineering-spark_itvdflab            "/deploy.sh"             4 days ago     Up 32 minutes                 0.0.0.0:8888->8888/tcp, :::8888->8888/tcp   data-engineering-spark_itvdflab_1
825d100aa0c4   postgres:13                                "docker-entrypoint.s…"   4 days ago     Up 34 minutes                 0.0.0.0:5432->5432/tcp, :::5432->5432/tcp   data-engineering-spark_pg.itversity.com_1
6a8dfded2af8   alpine:latest                              "/bin/sh"                6 months ago   Exited (137) 6 months ago                                                 mannul-container-build
2b6d386f5ded   hello-world                                "/bin/sh"                6 months ago   Created                                                                   gifted_wu
89a32b108df1   hello-world                                "/hello"                 6 months ago   Exited (0) 6 months ago                                                   wizardly_robinson
d68f9404d1fc   hello-world                                "sh"                     6 months ago   Created                                                                   vibrant_villani
fecd0ac4cb66   hello-world                                "/hello"                 6 months ago   Exited (0) 6 months ago                                                   happy_lumiere
cc9c6920c488   busybox                                    "sh"                     6 months ago   Exited (0) 6 months ago                                                   magical_blackburn
fadcf6f52f8d   597ce1600cf4                               "sh"                     6 months ago   Exited (137) 6 months ago                                                 naughty_burnell
e2dfddf2a82a   tensorflow/tensorflow:latest-gpu-jupyter   "bash -c 'source /et…"   7 months ago   Exited (127) 7 months ago     0.0.0.0:8888->8888/tcp, :::8888->8888/tcp   hardcore_elion
[tharindu@acer-lap data-engineering-spark]$ docker start my_pg
my_pg
[tharindu@acer-lap data-engineering-spark]$ docker ps
CONTAINER ID   IMAGE                             COMMAND                  CREATED      STATUS          PORTS                                       NAMES
c68b18841ca0   postgres                          "docker-entrypoint.s…"   2 days ago   Up 9 seconds    0.0.0.0:9999->5432/tcp, :::9999->5432/tcp   my_pg
8dec6c8c1b48   data-engineering-spark_itvdflab   "/deploy.sh"             4 days ago   Up 33 minutes   0.0.0.0:8888->8888/tcp, :::8888->8888/tcp   data-engineering-spark_itvdflab_1
825d100aa0c4   postgres:13                       "docker-entrypoint.s…"   4 days ago   Up 34 minutes   0.0.0.0:5432->5432/tcp, :::5432->5432/tcp   data-engineering-spark_pg.itversity.com_1
[tharindu@acer-lap data-engineering-spark]$ docker cp data/sms_db/users.csv my_pg:/tmp
[tharindu@acer-lap data-engineering-spark]$ docker exec -it my_pg ls -ltr /tmp
total 4
-rw-rw-r-- 1 1000 1000 319 Apr 21 11:40 users.csv
[tharindu@acer-lap data-engineering-spark]$
[tharindu@acer-lap data-engineering-spark]$
[tharindu@acer-lap data-engineering-spark]$ docker exec -it itv_pg psql -U postgres
Error: No such container: itv_pg
[tharindu@acer-lap data-engineering-spark]$ docker exec -it my_pg psql -U postgres
psql (14.2 (Debian 14.2-1.pgdg110+1))
Type "help" for help.

postgres=#
postgres=#
postgres=# CREATE DATABASE itversity_sms_db;
CREATE DATABASE
postgres=# CREATE USER tharindu WITH PASSWORD 'tharindu@123';
CREATE ROLE
postgres=# GRANT ALL ON DATABASE itversity_sms_db TO tharindu;
GRANT
postgres=# GRANT pg_read_server_files TO tharindu;
GRANT ROLE
postgres=# exit
[tharindu@acer-lap data-engineering-spark]$ psql -U tharindu -h localhost -p 9999 -d itversity_sms_db -w
psql: error: fe_sendauth: no password supplied
[tharindu@acer-lap data-engineering-spark]$ psql -U tharindu -h localhost -p 9999 -d itversity_sms_db -W
Password:
psql (12.9 (Ubuntu 12.9-0ubuntu0.20.04.1), server 14.2 (Debian 14.2-1.pgdg110+1))
WARNING: psql major version 12, server major version 14.
         Some psql features might not work.
Type "help" for help.

itversity_sms_db=>
itversity_sms_db=>
itversity_sms_db=>
itversity_sms_db=>
itversity_sms_db=> \l
                                    List of databases
       Name       |  Owner   | Encoding |  Collate   |   Ctype    |   Access privileges
------------------+----------+----------+------------+------------+-----------------------
 itversity_sms_db | postgres | UTF8     | en_US.utf8 | en_US.utf8 | =Tc/postgres         +
                  |          |          |            |            | postgres=CTc/postgres+
                  |          |          |            |            | tharindu=CTc/postgres
 postgres         | postgres | UTF8     | en_US.utf8 | en_US.utf8 |
 template0        | postgres | UTF8     | en_US.utf8 | en_US.utf8 | =c/postgres          +
                  |          |          |            |            | postgres=CTc/postgres
 template1        | postgres | UTF8     | en_US.utf8 | en_US.utf8 | =c/postgres          +
                  |          |          |            |            | postgres=CTc/postgres
(4 rows)

itversity_sms_db=> \d
Did not find any relations.
itversity_sms_db=> CREATE TABLE users (
itversity_sms_db(>     user_id SERIAL PRIMARY KEY,
itversity_sms_db(>     user_first_name VARCHAR(30) NOT NULL,
itversity_sms_db(>     user_last_name VARCHAR(30) NOT NULL,
itversity_sms_db(>     user_email_id VARCHAR(50) NOT NULL,
itversity_sms_db(>     user_email_validated BOOLEAN DEFAULT FALSE,
itversity_sms_db(>     user_password VARCHAR(200),
itversity_sms_db(>     user_role VARCHAR(1) NOT NULL DEFAULT 'U', --U and A
itversity_sms_db(>     is_active BOOLEAN DEFAULT FALSE,
itversity_sms_db(>     created_dt DATE DEFAULT CURRENT_DATE
itversity_sms_db(> );
CREATE TABLE
itversity_sms_db=>
itversity_sms_db=>
itversity_sms_db=> \d
                List of relations
 Schema |       Name        |   Type   |  Owner
--------+-------------------+----------+----------
 public | users             | table    | tharindu
 public | users_user_id_seq | sequence | tharindu
(2 rows)

itversity_sms_db=>
itversity_sms_db=>
itversity_sms_db=>
itversity_sms_db=>
itversity_sms_db=> /d users
itversity_sms_db-> \d users
                                             Table "public.users"
        Column        |          Type          | Collation | Nullable |                Default
----------------------+------------------------+-----------+----------+----------------------------------------
 user_id              | integer                |           | not null | nextval('users_user_id_seq'::regclass)
 user_first_name      | character varying(30)  |           | not null |
 user_last_name       | character varying(30)  |           | not null |
 user_email_id        | character varying(50)  |           | not null |
 user_email_validated | boolean                |           |          | false
 user_password        | character varying(200) |           |          |
 user_role            | character varying(1)   |           | not null | 'U'::character varying
 is_active            | boolean                |           |          | false
 created_dt           | date                   |           |          | CURRENT_DATE
Indexes:
    "users_pkey" PRIMARY KEY, btree (user_id)

itversity_sms_db->
itversity_sms_db->
itversity_sms_db-> \d users_user_id_seq
                 Sequence "public.users_user_id_seq"
  Type   | Start | Minimum |  Maximum   | Increment | Cycles? | Cache
---------+-------+---------+------------+-----------+---------+-------
 integer |     1 |       1 | 2147483647 |         1 | no      |     1
Owned by: public.users.user_id

itversity_sms_db->
itversity_sms_db->
itversity_sms_db-> COPY users(user_first_name, user_last_name,
itversity_sms_db(>     user_email_id, user_role, created_dt
itversity_sms_db(> ) FROM '/tmp/users.csv'
itversity_sms_db-> DELIMITER ','
itversity_sms_db-> CSV HEADER;
ERROR:  syntax error at or near "/"
LINE 1: /d users
        ^
itversity_sms_db=> COPY users(user_first_name, user_last_name,
    user_email_id, user_role, created_dt
) FROM '/tmp/users.csv'
DELIMITER ','
CSV HEADER;
COPY 5
itversity_sms_db=>
itversity_sms_db=>
itversity_sms_db=> SELECT * FROM users;
 user_id | user_first_name | user_last_name |        user_email_id         | user_email_validated | user_password | user_role | is_active | created_dt
---------+-----------------+----------------+------------------------------+----------------------+---------------+-----------+-----------+------------
       1 | Gordan          | Bradock        | gbradock0@barnesandnoble.com | f                    |               | A         | f         | 2020-01-10
       2 | Tobe            | Lyness         | tlyness1@paginegialle.it     | f                    |               | U         | f         | 2020-02-10
       3 | Addie           | Mesias         | amesias2@twitpic.com         | f                    |               | U         | f         | 2020-03-05
       4 | Corene          | Kohrsen        | ckohrsen3@buzzfeed.com       | f                    |               | U         | f         | 2020-04-15
       5 | Darill          | Halsall        | dhalsall4@intel.com          | f                    |               | U         | f         | 2020-10-10
(5 rows)

itversity_sms_db=>
itversity_sms_db=>
itversity_sms_db=>
itversity_sms_db=>

```